@ Author: ZHANG Mofan

@ Tsinghua University & Ecole Centrale de Lyon & Ecole des ponts ParisTech

@ Date: 07/05/2020

Dice Trading

In [29]:
#1.(b) Code
import numpy as np
import random
import math

def heuristicStrategy(t, x):
    if t<5 and x>=6:
        return True
#True=buy a new dice

In [30]:
#1.(c)
# Simulator Monte-Carlo
def Simulator(pi):
    nbMC = 1000
    C = [] # Cost. Here is the gain (total points at turn 10)
    for i in range(nbMC):
        x = [0]*11
        ksi = [0]
        nbDices = 1
        for t in range(1, 11):
            ksi.append(max(random.randint(1,6) for k in range(nbDices)))
            if pi(t, x[t]) and (nbDices<3):
                nbDices+=1
                x[t] = x[t-1]-5+ksi[t]
            else:
                x[t] = x[t-1]+ksi[t]
        C.append(x[-1])
    
    #C_mean = np.mean(C)
    #C_SD = np.std(C, ddof = 1)
    return C, nbMC

In [31]:
#1.(c)bis
C,nbMC=Simulator(heuristicStrategy)
C_mean = np.mean(C)
C_SD = np.std(C, ddof = 1)
#95% confidence interval
lowerBound = C_mean-1.96*C_SD/math.sqrt(nbMC)
upperBound = C_mean+1.96*C_SD/math.sqrt(nbMC)

In [32]:
#1.(c)ter
print("95% confidence interval:")
print("["+str(lowerBound)+", "+str(upperBound)+"]")

95% confidence interval:
[34.465168038535936, 35.11283196146407]


In [33]:
#1.(d)bis
def lawDice(nbDices):
    k=np.array([1,2,3,4,5,6])
    P=(np.power(k,nbDices)-np.power(k-1,nbDices))/pow(6,nbDices)
    return P

In [34]:
#1.(e)
def diceTradingDP():
    nbDices=3
    totalP=61
    T=10
    V=np.zeros([nbDices,totalP,T+1]) # [nbDices, points, turns]
    strategyMap=np.zeros([nbDices,totalP,T+1]) # whether or not buying one more dice?
    
    P=[lawDice(1),lawDice(2),lawDice(3)] # law of the maximum of respectively 1, 2 and 3 dices
    
    for i in range(nbDices):
        for x in range(totalP):
            V[i,x,T]=x # the value at turn==10 is the points we have
    for t in range(T-1,0,-1):
        for x in range(6,totalP-6*(T-t)):
            # if we have already 3 dices            
            V[2,x,t]=sum(P[2]*V[2,x+1:x+7,t+1])
            
            # if we have now 2 dices (after dice roll of this turn)
            # temp1: value if buy one more dice
            # temp2: value if not buy one more dice
            temp1=sum(P[2]*V[2,x-4:x+2,t+1])
            temp2=sum(P[1]*V[1,x+1:x+7,t+1])
            if temp1>=temp2 and x>=6:
                V[1,x,t]=temp1
                strategyMap[1,x,t]=-5
            else:
                V[1,x,t]=temp2
                strategyMap[1,x,t]=0
                
            # if we have only 1 dice (after dice roll of this turn)
            # temp3: value if buy one mor edice
            # temp4: value if not buy one more dice
            temp3=sum(P[1]*V[1,x-4:x+2,t+1])
            temp4=sum(P[0]*V[0,x+1:x+7,t+1])
            if temp3>=temp4 and x>=6:
                V[0,x,t]=temp3
                strategyMap[0,x,t]=-5
            else:
                V[0,x,t]=temp4
                strategyMap[0,x,t]=0
                
        # when points<6, we can not buy one more dice
        for x in range(0,6):
            V[2,x,t]=sum(P[2]*V[2,x+1:x+7,t+1])
            V[1,x,t]=sum(P[1]*V[1,x+1:x+7,t+1])
            V[0,x,t]=sum(P[0]*V[0,x+1:x+7,t+1])
    
    # turn 0: we have 1 dice and 0 point
    V[0,0,0]=sum(P[0]*V[0,1:7,1])
    return V, strategyMap

In [35]:
#1.(e)bis
VMap,strategyMap=diceTradingDP()
print("Question 1.(e) Finding the optimal value V0 and trategy pi*")
print("V0:")
print(VMap[0,0,0])
#print(strategyMap[0])

Question 1.(e) Finding the optimal value V0 and trategy pi*
V0:
37.62152777777778


In [36]:
#1.(f)
def SimulatorDP(pi):
    nbMC = 1000
    C = [] # Cost. Here is the gain (total points at turn 10)
    VMap, strategyMap=pi()
    for i in range(nbMC):
        x = [0]*11
        ksi = []
        nbDices = 1
        for t in range(1, 11):
            ksi.append(max(random.randint(1,6) for k in range(nbDices)))
            if strategyMap[nbDices-1,x[t-1]+ksi[t-1],t]==-5 and (nbDices<3) and (x[t-1]+ksi[t-1]>=6):
                nbDices+=1
                x[t] = x[t-1]-5+ksi[t-1]
            else:
                x[t] = x[t-1]+ksi[t-1]
        C.append(x[-1])
    
    #C_mean = np.mean(C)
    #C_SD = np.std(C, ddof = 1)
    return C, nbMC

In [48]:
#1.(f)bis
C,nbMC=SimulatorDP(diceTradingDP)
C_mean = np.mean(C)
print("V0 mean:")
print(C_mean)
C_SD = np.std(C, ddof = 1)
#95% confidence interval
lowerBound = C_mean-1.96*C_SD/math.sqrt(nbMC)
upperBound = C_mean+1.96*C_SD/math.sqrt(nbMC)
print("95% confidence interval:")
print("["+str(lowerBound)+", "+str(upperBound)+"]")

V0 mean:
37.586
95% confidence interval:
[37.279970097821426, 37.89202990217857]


In [38]:
#2.(a)
# Sometimes we use "sell" instead of "loose"
def diceTradingDP_2a():
    nbDices=5
    totalP=120
    diceRoll=6
    T=10
    V=np.zeros([nbDices,totalP,diceRoll,T]) # [nbDices, points, turns]
    strategyBuyMap=np.zeros([nbDices,totalP,diceRoll,T]) # whether or not buying one more dice?
    strategySellMap=np.zeros([nbDices,totalP,diceRoll,T]) # whether or not selling one dice to double the points?
    
    P=[lawDice(1),lawDice(2),lawDice(3),lawDice(4),lawDice(5)] # law of the maximum of respectively 1, 2, 3, 4 and 5 dices
    
    # Last turn
    # have 1 dice
    for x in range(totalP):
        for k in range(diceRoll):
            V[0,x,k,-1]=x+k+1
    # have more than 1 dice, we choose to loose 1 and double the dice roll of last turn
    for i in range(1,nbDices):
        for x in range(totalP):
            for k in range(diceRoll):
                V[i,x,k,-1]=x+(k+1)*2
                strategySellMap[i,x,k,-1]=1
    
    for t in range(T-2,-1,-1):
        for x in range(totalP-12):
            for k in range(diceRoll):
                # have only one dice
                # if not buy one more dice
                temp1=sum(P[0]*V[0,x+k+1,0:6,t+1])
                # if buy one more dice
                temp2=sum(P[1]*V[1,x+k-4,0:6,t+1])
                if temp1>=temp2 or (x+k+1)<6:
                    V[0,x,k,t]=temp1
                else:
                    V[0,x,k,t]=temp2
                    strategyBuyMap[0,x,k,t]=-5
                
                # have 2, 3 or 4 dices
                for n in range(1,4):
                    # do nothing
                    temp3=sum(P[n]*V[n,x+k+1,0:6,t+1])
                    # loose one to double the dice roll of this turn
                    temp4=sum(P[n-1]*V[n-1,x+(k+1)*2,0:6,t+1])
                    # not loose 1, but buy 1
                    temp5=sum(P[n+1]*V[n+1,x+k-4,0:6,t+1])
                    # loose 1 and then buy 1
                    temp6=sum(P[n]*V[n,x+(k+1)*2-5,0:6,t+1])
                    if (x+k+1)>=6:
                        choice=np.argmax([temp3,temp4,temp5,temp6])
                    elif (x+(k+1)*2)>=6:
                        choice=np.argmax([temp3,temp4,0,temp6])
                    else:
                        choice=np.argmax([temp3,temp4,0,0])
                    if choice == 0:
                        V[n,x,k,t]=temp3
                    elif choice == 1:
                        V[n,x,k,t]=temp4
                        strategySellMap[n,x,k,t]=1
                    elif choice == 2:
                        V[n,x,k,t]=temp5
                        strategyBuyMap[n,x,k,t]=-5
                    else:
                        V[n,x,k,t]=temp6
                        strategySellMap[n,x,k,t]=1
                        strategyBuyMap[n,x,k,t]=-5
                
                
                # have 5 dices
                # do nothing
                temp7=sum(P[4]*V[4,x+k+1,0:6,t+1])
                # loose one
                temp8=sum(P[3]*V[3,x+(k+1)*2,0:6,t+1])
                # loose one and buy one
                temp9=sum(P[4]*V[4,x+(k+1)*2-5,0:6,t+1])
                if (x+(k+1)*2)>=6:
                    choice=np.argmax([temp7,temp8,temp9])
                else:
                    choice=np.argmax([temp7,temp8,0])
                if choice == 0:
                    V[4,x,k,t]=temp7
                elif choice == 1:
                    V[4,x,k,t]=temp8
                    strategySellMap[4,x,k,t]=1
                elif choice == 2:
                    V[4,x,k,t]=temp9
                    strategySellMap[4,x,k,t]=1
                    strategyBuyMap[4,x,k,t]=-5
    
    return V, strategySellMap, strategyBuyMap

In [39]:
#2.(a)bis
VMap, strategySellMap, strategyBuyMap = diceTradingDP_2a()
V0=(VMap[0,0,0,0]+VMap[0,0,1,0]+VMap[0,0,2,0]+VMap[0,0,3,0]+VMap[0,0,4,0]+VMap[0,0,5,0])/6
print("V0:")
print(V0)

V0:
47.59380284322564


In [40]:
#2.(a)ter
def SimulatorDP_2a(pi):
    nbMC = 10000
    C = [] # Cost. Here is the gain (total points at turn 10)
    VMap, strategySellMap, strategyBuyMap=pi()
    for i in range(nbMC):
        x_temp=0
        x = [0]
        ksi = []
        nbDices = 1
        
        for t in range(10):
            ksi.append(max(random.randint(1,6) for k in range(nbDices)))
            x_temp=x[-1]+ksi[-1]
            choiceSell=strategySellMap[nbDices-1,x[-1],ksi[-1]-1,t]
            choiceBuy=strategyBuyMap[nbDices-1,x[-1],ksi[-1]-1,t]
            if choiceSell==1:
                nbDices-=1
                x_temp+=ksi[-1]
            if choiceBuy==-5:
                nbDices+=1
                x_temp-=5
            x.append(x_temp)

        C.append(x[-1])
    
    #C_mean = np.mean(C)
    #C_SD = np.std(C, ddof = 1)
    return C, nbMC

In [50]:
#2.(a)quater
C,nbMC=SimulatorDP_2a(diceTradingDP_2a)
C_mean = np.mean(C)
print("V0 mean:")
print(C_mean)
C_SD = np.std(C, ddof = 1)
#95% confidence interval
lowerBound = C_mean-1.96*C_SD/math.sqrt(nbMC)
upperBound = C_mean+1.96*C_SD/math.sqrt(nbMC)
print("95% confidence interval:")
print("["+str(lowerBound)+", "+str(upperBound)+"]")

V0 mean:
47.5278
95% confidence interval:
[47.39711447688293, 47.658485523117065]


In [42]:
#2.(b)
# We introduce here one support method to facilitate the question
def expectationValue(V,strategySellMap,strategyBuyMap,nbDices,x,k,t):
    nbDices+=1
    ksi=np.array([1,2,3,4,5,6])
    P1=(np.power(ksi,nbDices) - np.power(ksi-1,nbDices))/np.power(6,nbDices)
    P2=(np.power(ksi,nbDices-1) - np.power(ksi-1,nbDices-1))/np.power(6,nbDices-1)
    P3=(np.power(ksi,nbDices+1) - np.power(ksi-1,nbDices+1))/np.power(6,nbDices+1)
    #Do nothing
    temp1=(P1[0]*V[nbDices-1,x+k+1,0,t+1]+P1[1]*V[nbDices-1,x+k+1,1,t+1]+P1[2]*V[nbDices-1,x+k+1,2,t+1]+P1[3]*V[nbDices-1,x+k+1,3,t+1]+P1[4]*V[nbDices-1,x+k+1,4,t+1]+P1[5]*V[nbDices-1,x+k+1,5,t+1])
    #Sell one dice
    temp2=(P2[0]*V[nbDices-2,x+(k+1)*2,0,t+1]+P2[1]*V[nbDices-2,x+(k+1)*2,1,t+1]+P2[2]*V[nbDices-2,x+(k+1)*2,2,t+1]+P2[3]*V[nbDices-2,x+(k+1)*2,3,t+1]+P2[4]*V[nbDices-2,x+(k+1)*2,4,t+1]+P2[5]*V[nbDices-2,x+(k+1)*2,5,t+1])
    #Buy one dice
    temp3=(P3[0]*V[nbDices,x+k-4,0,t+1]+P3[1]*V[nbDices,x+k-4,1,t+1]+P3[2]*V[nbDices,x+k-4,2,t+1]+P3[3]*V[nbDices,x+k-4,3,t+1]+P3[4]*V[nbDices,x+k-4,4,t+1]+P3[5]*V[nbDices,x+k-4,5,t+1])
    #Sell one and then buy one
    temp4=(P1[0]*V[nbDices-1,x+(k+1)*2-5,0,t+1]+P1[1]*V[nbDices-1,x+(k+1)*2-5,1,t+1]+P1[2]*V[nbDices-1,x+(k+1)*2-5,2,t+1]+P1[3]*V[nbDices-1,x+(k+1)*2-5,3,t+1]+P1[4]*V[nbDices-1,x+(k+1)*2-5,4,t+1]+P1[5]*V[nbDices-1,x+(k+1)*2-5,5,t+1])
    if (x+k+1)>=6:
        choice=np.argmax([temp1,temp2,temp3,temp4])
    elif (x+(k+1)*2)>=6:
        choice=np.argmax([temp1,temp2,0,temp4])
    else:
        choice=np.argmax([temp1,temp2,0,0])
    if choice == 0:
        V[nbDices-1,x,k,t]=temp1
    elif choice == 1:
        V[nbDices-1,x,k,t]=temp2
        strategySellMap[nbDices-1,x,k,t]=1
    elif choice == 2:
        V[nbDices-1,x,k,t]=temp3
        strategyBuyMap[nbDices-1,x,k,t]=-5
    else:
        V[nbDices-1,x,k,t]=temp4
        strategySellMap[nbDices-1,x,k,t]=1
        strategyBuyMap[nbDices-1,x,k,t]=-5
    return V,strategySellMap,strategyBuyMap
    
    
def diceTradingDP_2b():
    nbDices=10
    totalP=120
    diceRoll=6
    T=10
    V=np.zeros([nbDices,totalP,diceRoll,T]) # [nbDices, points, turns]
    strategyBuyMap=np.zeros([nbDices,totalP,diceRoll,T]) # whether or not buying one more dice?
    strategySellMap=np.zeros([nbDices,totalP,diceRoll,T]) # whether or not selling one dice to double the points?
    
    # Last turn
    # have 1 dice
    for x in range(totalP):
        for k in range(diceRoll):
            V[0,x,k,-1]=x+k+1
    # have more than 1 dice, we choose to loose 1 and double the dice roll of last turn
    for i in range(1,nbDices):
        for x in range(totalP):
            for k in range(diceRoll):
                V[i,x,k,-1]=x+(k+1)*2
                strategySellMap[i,x,k,-1]=1
    
    for t in range(T-2,-1,-1):
        for x in range(totalP-12):
            for k in range(diceRoll):
                # have only one dice
                # if not buy one more dice
                temp1=(V[0,x+k+1,0,t+1]+V[0,x+k+1,1,t+1]+V[0,x+k+1,2,t+1]+V[0,x+k+1,3,t+1]+V[0,x+k+1,4,t+1]+V[0,x+k+1,5,t+1])/6
                # if buy one more dice
                temp2=(V[1,x+k-4,0,t+1]+3*V[1,x+k-4,1,t+1]+5*V[1,x+k-4,2,t+1]+7*V[1,x+k-4,3,t+1]+9*V[1,x+k-4,4,t+1]+11*V[1,x+k-4,5,t+1])/36
                if temp1>=temp2 or (x+k+1)<6:
                    V[0,x,k,t]=temp1
                else:
                    V[0,x,k,t]=temp2
                    strategyBuyMap[0,x,k,t]=-5
                
                for i in range(1,nbDices-1):
                    # have more than 1 dice
                    V,strategySellMap,strategyBuyMap=expectationValue(V,strategySellMap,strategyBuyMap,i,x,k,t)
    
    return V, strategySellMap, strategyBuyMap

In [43]:
#2.(b)bis
VMap, strategySellMap, strategyBuyMap = diceTradingDP_2b()
V0=(VMap[0,0,0,0]+VMap[0,0,1,0]+VMap[0,0,2,0]+VMap[0,0,3,0]+VMap[0,0,4,0]+VMap[0,0,5,0])/6
print("Question 2.(b): What happen if we do not restrict the maximum number of dices that can be owned by a player?")
print("V0:")
print(V0)

Question 2.(b): What happen if we do not restrict the maximum number of dices that can be owned by a player?
V0:
47.594134977086675


In [44]:
#2.(c)
def diceTradingDP_2c():
    nbDices=5
    totalP=120
    diceRoll=6
    T=10
    V=np.zeros([nbDices,totalP,diceRoll,T]) # [nbDices, points, turns]
    strategyBuyMap=np.zeros([nbDices,totalP,diceRoll,T]) # whether or not buying one more dice?
    strategySellMap=np.zeros([nbDices,totalP,diceRoll,T]) # whether or not selling one dice to double the points?
    
    P=[lawDice(1),lawDice(2),lawDice(3),lawDice(4),lawDice(5)] # law of the maximum of respectively 1, 2, 3, 4 and 5 dices
    
    sellPrices=[0,2,4,5,8]
    
    # Last turn
    # have 1 dice
    for x in range(totalP):
        for k in range(diceRoll):
            V[0,x,k,-1]=x+k+1
    # have 2 or more dices
    for x in range(totalP):
        for k in range(diceRoll):
            for n in range(1,5):
                tempLT1=k+1+sellPrices[n]
                tempLT2=(k+1)*2+sellPrices[n-1]
                if tempLT1>=tempLT2:
                    V[n,x,k,-1]=x+tempLT1
                else:
                    V[n,x,k,-1]=x+tempLT2
    
    for t in range(T-2,-1,-1):
        for x in range(totalP-12):
            for k in range(diceRoll):
                # have only one dice
                # if not buy one more dice
                temp1=sum(P[0]*V[0,x+k+1,0:6,t+1])
                # if buy one more dice
                temp2=sum(P[1]*V[1,x+k-4,0:6,t+1])
                if temp1>=temp2 or (x+k+1)<6:
                    V[0,x,k,t]=temp1
                else:
                    V[0,x,k,t]=temp2
                    strategyBuyMap[0,x,k,t]=-5
                
                # have 2, 3 or 4 dices
                for n in range(1,4):
                    # do nothing
                    temp3=sum(P[n]*V[n,x+k+1,0:6,t+1])
                    # loose one to double the dice roll of this turn
                    temp4=sum(P[n-1]*V[n-1,x+(k+1)*2,0:6,t+1])
                    # not loose 1, but buy 1
                    temp5=sum(P[n+1]*V[n+1,x+k-4,0:6,t+1])
                    # loose 1 and then buy 1
                    temp6=sum(P[n]*V[n,x+(k+1)*2-5,0:6,t+1])
                    if (x+k+1)>=6:
                        choice=np.argmax([temp3,temp4,temp5,temp6])
                    elif (x+(k+1)*2)>=6:
                        choice=np.argmax([temp3,temp4,0,temp6])
                    else:
                        choice=np.argmax([temp3,temp4,0,0])
                    if choice == 0:
                        V[n,x,k,t]=temp3
                    elif choice == 1:
                        V[n,x,k,t]=temp4
                        strategySellMap[n,x,k,t]=1
                    elif choice == 2:
                        V[n,x,k,t]=temp5
                        strategyBuyMap[n,x,k,t]=-5
                    else:
                        V[n,x,k,t]=temp6
                        strategySellMap[n,x,k,t]=1
                        strategyBuyMap[n,x,k,t]=-5
                
                # have 5 dices
                # do nothing
                temp7=sum(P[4]*V[4,x+k+1,0:6,t+1])
                # loose one
                temp8=sum(P[3]*V[3,x+(k+1)*2,0:6,t+1])
                # loose one and buy one
                temp9=sum(P[4]*V[4,x+(k+1)*2-5,0:6,t+1])
                if (x+(k+1)*2)>=6:
                    choice=np.argmax([temp7,temp8,temp9])
                else:
                    choice=np.argmax([temp7,temp8,0])
                if choice == 0:
                    V[4,x,k,t]=temp7
                elif choice == 1:
                    V[4,x,k,t]=temp8
                    strategySellMap[4,x,k,t]=1
                elif choice == 2:
                    V[4,x,k,t]=temp9
                    strategySellMap[4,x,k,t]=1
                    strategyBuyMap[4,x,k,t]=-5
    
    return V, strategySellMap, strategyBuyMap

In [45]:
#2.(c)bis
VMap, strategySellMap, strategyBuyMap = diceTradingDP_2c()
V0=(VMap[0,0,0,0]+VMap[0,0,1,0]+VMap[0,0,2,0]+VMap[0,0,3,0]+VMap[0,0,4,0]+VMap[0,0,5,0])/6
print("Question 2.(c): At the end of the game, the remaining dices are sold to the next player.")
print("V0:")
print(V0)

Question 2.(c): At the end of the game, the remaining dices are sold to the next player.
V0:
47.71271284296862
